# Wren Engine Demo

This notebook demonstrates the basic usage of the Wren Engine Python API in local environment.

## Import Required Packages

In [ ]:
from app.model.data_source import DataSource
from wren import create_session_context

## Configure Data Source
Currently supported data sources:
- athena
- bigquery
- canner
- clickhouse
- mssql
- mysql
- oracle
- postgres
- redshift
- snowflake
- trino
- local_file
- s3_file
- minio_file
- gcs_file

Refer to [ConnectionInfo](https://docs.getwren.ai/oss/wren_engine_api#tag/AthenaConnectionInfo) for available parameters.

In [ ]:
# Configure local file data source
data_source = DataSource.local_file

# Set connection information
info = data_source.get_connection_info(
    {
        "url": "../resources/demo",
        "format": "duckdb",
    }
)

print(f"Data source type: {data_source}")
print(f"Connection info: {info}")

## Create Wren Session

In [ ]:
# Create session context
session = create_session_context(
    data_source="local_file",
    connection_info=info,
    mdl_path="../resources/demo/jaffle_shop_mdl.json",
)

print("Session created successfully!")

## Execute Query

In [ ]:
# Query customer data
task = session.sql("select * from customers").execute()
print("Show planned SQL: ", task.planned_sql)
print("Show dialect SQL: ", task.dialect_sql)
print("Show the arrow table of the result: \n", task.results)
print("Show the formatted JSON result: \n", task.formatted_result())

# Execute Dry Run

In [ ]:
# Query first 10 customer records
task = session.sql("select * from customers limit 10")
task.dry_run()
print("Show planned SQL: ", task.planned_sql)
print("Show dialect SQL: ", task.dialect_sql)
print("Dry run executed successfully!")

# Access Control

## Row-Level Access Control
In `resources/demo/jaffle_shop_mdl.json`, we set `status_rule` for `orders`.
```json
{
    "name": "status_rule",
    "requiredProperties": [
        {
            "name": "session_status",
            "required": true
        }
    ],
    "condition": "status = @session_status"
}
```
Therefore, we can use `session_status` for access control. By specifying the value of `X-Wren-Variable-Session_Status` in Session Properties, we can control data visibility. `X-Wren-Variable-` is the prefix for Wren Engine Session Properties.


In [ ]:
# Set Session Properties
properties = {"X-Wren-Variable-Session_Status": "'completed'"}
task = session.sql("select * from orders limit 10", properties).execute()
print("Show planned SQL: \n", task.planned_sql)
print("Show dialect SQL: \n", task.dialect_sql)
print("Show the arrow table of the result: \n", task.results)
print("Show the formatted JSON result: \n", task.formatted_result())

## Column-Level Access Control
In `resources/demo/jaffle_shop_mdl.json`, we set the `credit_card_access` rule for `orders.credit_card_amount`.
```json
{
    "name": "credit_card_amount",
    "type": "double",
    "columnLevelAccessControl": {
        "name": "credit_card_access",
        "requiredProperties": [
            {
                "name": "session_level",
                "required": false
            }
        ],
        "operator": "GREATER_THAN",
        "threshold": "3"
    },
    "properties": {
    "description": "Amount of the order (AUD) paid for by credit card"
    }
},
```
When `session_level` is greater than `3`, `credit_card_amount` can be accessed.


In [ ]:
# credit_card_amount can be accessed when Session_Level is greater than or equal to 3
properties = {"X-Wren-Variable-Session_Level": "4"}
task = session.sql(
    "select credit_card_amount from orders limit 10", properties
).execute()
print("Show planned SQL: ", task.planned_sql)
print("Show dialect SQL: ", task.dialect_sql)
print("Show the arrow table of the result: \n", task.results)
print("Show the formatted JSON result: \n", task.formatted_result())

In [ ]:
# credit_card_amount can't be accessed when Session_Level is less than 3
try:
    properties = {"X-Wren-Variable-Session_Level": "1"}
    task = session.sql(
        "select credit_card_amount from orders limit 10", properties
    ).execute()
except Exception as e:
    print("Error occurred:", e)

## Data Visualization with Pandas (Optional)
`Task.results` is a `pyarrow.Table` that can be converted to commonly used DataFrame through `to_pandas` or other APIs for further processing.

In [ ]:
task = session.sql("select * from customers limit 10").execute()
# Assuming result has a method to convert to DataFrame
df = task.results.to_pandas()  # This method may need adjustment
print("DataFrame preview:")
print(df.head())